# Setup

In [7]:
# setup including building the graph. has the side effect of basic validation of turtle files.
import rdflib
import os
import dominate
from dominate.tags import *

g = rdflib.Graph()

# directory paths assume existence of the p-lod dir two levels up
result = g.parse("../../p-lod/p-lod-vocabulary.ttl", format="turtle")
result = g.parse("../../p-lod/p-lod-numbered-entities.ttl", format="turtle")
result = g.parse("../../p-lod/p-lod-named-entities.ttl", format="turtle")

ns = {"dcterms": "http://purl.org/dc/terms/",
      "rdf" : "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
      "rdfs": "http://www.w3.org/2000/01/rdf-schema#" ,
      "p-lod" : "http://digitalhumanities.umass.edu/p-lod/",
      "p-lod-v" : "http://digitalhumanities.umass.edu/p-lod/vocabulary/",
      "p-lod-e" : "http://digitalhumanities.umass.edu/p-lod/entities/"}

# Build individual files

In [8]:
# get unique subjects
subjects = g.query(
    """SELECT DISTINCT ?s ?label
       WHERE {
          ?s ?p ?o .
          ?s rdfs:label ?label .
          ?s rdf:type ?type .
       } ORDER BY ?s""", initNs = ns)

# build individual index.html files
for row in subjects:
    path = str(row.s).replace('http://digitalhumanities.umass.edu/p-lod/','').split('/')
    
    directory = path[0]
    entity = path[1]
    
    if not os.path.exists("../" + directory):
        os.makedirs("../" + directory)
        
    if not os.path.exists("../" + directory + "/" + entity):
        os.makedirs("../" + directory + "/" + entity)
        
    p_os = g.query(
        """SELECT ?p ?o ?plabel ?olabel WHERE {
           <http://digitalhumanities.umass.edu/p-lod/""" + directory + "/" + entity + """> ?p ?o .
           OPTIONAL {?p rdfs:label ?plabel} 
           OPTIONAL {?o rdfs:label ?olabel} 
           } ORDER BY ?p""", initNs = ns)
    
    doc = dominate.document(title="Pompeii LOD: " + entity)

    with doc.head:
        link(rel='stylesheet', href='../pure-min.css')
        link(rel='stylesheet', href='../../pure-min.css')
        # script(type='text/javascript', src='script.js')

    with doc:
        h1("Pompeii LOD : " + row.label )
        p(a("(top)", href="../../"), style = "margin")
        for p_o in p_os:
            if str(p_o.p) == "http://www.w3.org/2000/01/rdf-schema#label":
                break
                
            if str(p_o.plabel) != 'None':
                p(str(p_o.plabel)+":", style = "margin-left:.25em")
            else:
                p(i(str(p_o.p)), style = "margin-left:.25em")
                
            with p(style="margin-left:1em"):
                if str(p_o.olabel) != "None":
                    olabel = str(p_o.olabel)
                else:
                    olabel = str(p_o.o)
                
                if str(p_o.o)[0:4] == 'http':
                    a(olabel,href = str(p_o.o).replace('http://digitalhumanities.umass.edu/p-lod',''))
                else:
                    span(olabel)
            p()

    # actually write the file
    file = open("../" + directory + "/" + entity + "/index.html", 'w')
    file.write(doc.render())
    file.close()

# Build main index.html file

In [3]:
# build /index.html
index = g.query(
    """SELECT DISTINCT ?s ?label
       WHERE {
          ?s ?p ?o .
          ?s rdfs:label ?label .
          ?s a ?type .
          ?type rdfs:subClassOf* p-lod-v:named-entity .
          OPTIONAL { ?s a p-lod-v:named-entity . }
       } ORDER BY ?s""", initNs = ns)

doc = dominate.document(title="Pompeii LOD: ")

with doc.head:
    link(rel='stylesheet', href='pure-min.css')
    # script(type='text/javascript', src='script.js')

with doc:
    h1("Pompeii LOD")
    with ol():
        for i in index:
            with li():
                a(str(i.label),href = str(i.s).replace('http://digitalhumanities.umass.edu/p-lod',''))

# actually write the file
file = open("../index.html", 'w')
file.write(doc.render())
file.close()